In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
spark=SparkSession.builder.master("local[2]").appName("log parser").getOrCreate()
sc=spark.sparkContext

In [2]:
data=sc.textFile("/user/edureka_1152044/access",4)

In [168]:
#help(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
data=data.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)

In [5]:
rdd=sc.parallelize(data.take(100000),6)

In [6]:
rdd_split=rdd.map(lambda x: x.split("\n"))

# Find out how many 404 HTTP codes are in access logs

In [34]:
import re
def func(rdd):
    line=None
    if re.search("HTTP/1.1 404 ",rdd[0]):
        line=rdd[0]
    return line
rdd_split.map(func).filter(lambda x:(x!=None)).count()

3844

# Find out which URLs are broken

In [46]:
import re
def func(rdd):
    line=None
    if re.search("http://[\w\S]+[\s]",rdd):
        line=re.search("http://[\w\S]+[\s]",rdd).group(0)    
    return line
def extract(rdd):
    line=None
    if re.search("HTTP/1.1 404 ",rdd):
        line=rdd
    return line
rdd_split.map(lambda x:x[0]).filter(extract).map(func).collect()

[u'http://almhuette-raith.at/ ',
 u'http://www.almhuette-raith.at/ ',
 None,
 u'http://www.bing.com/bingbot.htm) ',
 u'http://www.almhuette-raith.at/ ',
 u'http://www.bing.com/bingbot.htm) ',
 u'http://www.bing.com/bingbot.htm) ',
 u'http://www.almhuette-raith.at/ ',
 None,
 u'http://almhuette-raith.at/ ',
 u'http://almhuette-raith.at/ ',
 u'http://almhuette-raith.at/ ',
 u'http://www.almhuette-raith.at/ ',
 u'http://www.almhuette-raith.at/ ',
 None,
 None,
 u'http://www.almhuette-raith.at/index.php?option=com_content&view=article&id=49&Itemid=55 ',
 u'http://www.almhuette-raith.at/index.php?option=com_content&view=article&id=46&Itemid=54 ',
 u'http://www.almhuette-raith.at/ ',
 u'http://almhuette-raith.at/ ',
 u'http://www.almhuette-raith.at/ ',
 u'http://www.almhuette-raith.at/ ',
 u'http://www.almhuette-raith.at/ ',
 u'http://almhuette-raith.at/ ',
 u'http://www.almhuette-raith.at/ ',
 None,
 u'http://www.almhuette-raith.at/index.php?option=com_content&view=article&id=49&Itemid=55 '

# Verify there are no null columns in the original dataset.

In [172]:
df=spark.read.text(r'/user/edureka_1152044/access')

In [173]:
df.show(3)

+--------------------+
|               value|
+--------------------+
|109.169.248.247 -...|
|109.169.248.247 -...|
|46.72.177.4 - - [...|
+--------------------+
only showing top 3 rows



In [174]:
from pyspark.sql.functions import col
df.where(col("value").isNull()).show()

+-----+
|value|
+-----+
+-----+



# Replace null values with constants such as 0

In [175]:
def func(rdd):
    line=rdd[0].replace("NULL","0").replace("null","0")
    return line
rdd_split.map(func).collect()

[u'109.169.248.247 - - [12/Dec/2015:18:25:11 +0100] GET /administrator/ HTTP/1.1 200 4263 - Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0 -',
 u'109.169.248.247 - - [12/Dec/2015:18:25:11 +0100] POST /administrator/index.php HTTP/1.1 200 4494 http://almhuette-raith.at/administrator/ Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0 -',
 u'46.72.177.4 - - [12/Dec/2015:18:31:08 +0100] GET /administrator/ HTTP/1.1 200 4263 - Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0 -',
 u'46.72.177.4 - - [12/Dec/2015:18:31:08 +0100] POST /administrator/index.php HTTP/1.1 200 4494 http://almhuette-raith.at/administrator/ Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0 -',
 u'83.167.113.100 - - [12/Dec/2015:18:31:25 +0100] GET /administrator/ HTTP/1.1 200 4263 - Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0 -',
 u'83.167.113.100 - - [12/Dec/2015:18:31:25 +0100] POST /administrator/index.php HTTP/1.1 200 4494 ht

In [176]:
from datetime import datetime
def func(rdd):
    pattern = r'\d\d/\w\w\w/\d\d\d\d'
    grp=re.search(pattern,rdd).group(0)
    date=datetime.strptime(grp,"%d/%b/%Y").date()
    date=date.strftime('%d %B %Y')
    return date
rdd_split.map(lambda x: x[0]).map(func).collect()

['12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December 2015',
 '12 December

# Parse timestamp to readable date

# Describe which HTTP status values appear in data and how many.

In [212]:
def func(rdd):
    line=None
    pattern = r' \d\d\d '
    if re.search(pattern,rdd):
        line=re.search(pattern,rdd).group(0)
        status=re.search(r"\d\d\d",line).group(0)
    return status
rdd_split.map(lambda x: x[0]).map(func).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).map(lambda x: (0,x[1]) if(x[0]==None) else (x[0],x[1])).collect()

[(u'500', 18),
 (u'200', 94687),
 (u'501', 1),
 (u'206', 136),
 (u'304', 784),
 (u'405', 6),
 (u'301', 85),
 (u'404', 4283)]

# Display as chart the above stat in chart in Zeppelin notebook

# How many unique hosts are there in theentire log and their average request

In [163]:
rdd_split.map(lambda x: ((x[0].split(" ")[0],x[0].split(" ")[5]),1)).reduceByKey(lambda x,y: x+y).collect()

[((u'178.150.142.246', u'POST'), 9),
 ((u'176.104.3.31', u'POST'), 6),
 ((u'213.234.14.220', u'POST'), 11),
 ((u'89.28.163.56', u'POST'), 3),
 ((u'217.66.152.18', u'POST'), 3),
 ((u'157.55.39.160', u'GET'), 1),
 ((u'37.215.77.225', u'POST'), 3),
 ((u'31.43.21.241', u'GET'), 3),
 ((u'5.129.198.96', u'POST'), 3),
 ((u'91.76.192.76', u'POST'), 3),
 ((u'95.220.191.118', u'GET'), 3),
 ((u'95.29.204.245', u'GET'), 1),
 ((u'178.158.114.185', u'GET'), 3),
 ((u'78.60.250.247', u'POST'), 3),
 ((u'176.51.228.62', u'GET'), 3),
 ((u'93.115.95.207', u'GET'), 1),
 ((u'178.187.59.219', u'POST'), 3),
 ((u'37.55.31.191', u'GET'), 3),
 ((u'31.173.242.0', u'GET'), 3),
 ((u'201.8.51.69', u'GET'), 6),
 ((u'78.111.187.172', u'POST'), 3),
 ((u'31.129.182.110', u'GET'), 3),
 ((u'178.90.248.81', u'GET'), 3),
 ((u'31.181.232.253', u'GET'), 3),
 ((u'93.74.20.107', u'GET'), 3),
 ((u'78.173.109.53', u'GET'), 3),
 ((u'95.24.193.81', u'POST'), 3),
 ((u'176.111.213.15', u'GET'), 3),
 ((u'87.249.198.130', u'POST'), 3),

In [220]:
unique_hosts=rdd_split.map(lambda x: (x[0].split(" ")[0],1)).reduceByKey(lambda x,y: x+y)
unique_count=unique_hosts.count()
total_hosts=unique_hosts.map(lambda x: x[1])
avg=float(sum(total_hosts.collect()))/float(unique_count)
print(float(sum(total_hosts.collect())),float(unique_count))
print("average of total number of hosts : ",avg)

(100000.0, 9705.0)
('average of total number of hosts : ', 10.303967027305513)
